### Fetching requisite data

In [0]:
import requests
import json
import pandas as pd
import numpy as np
import string
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

API_KEY = "" #insert your API key here

API_HOST = 'https://api.yelp.com'
BUSINESS_PATH = '/v3/businesses/'

[nltk_data] Downloading package punkt to C:\Users\Abhishek
[nltk_data]     Patria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
#Product Id of New York restaurants

ny = pd.read_csv('./YelpNYC//productIdMapping', sep='\t', header=None)
ny.columns = ['name', 'id']
ny = ny.assign(alias = ['not found' for i in range(len(ny.name))])
ny = ny.assign(latitude = [0.0 for i in range(len(ny.name))])
ny  = ny.assign(longitude = [0.0 for i in range(len(ny.name))])
ny  = ny.assign(zipcode = [0 for i in range(len(ny.name))])
ny  = ny.assign(review_count = [0.0 for i in range(len(ny.name))])
ny  = ny.assign(complete_name = ['not found' for i in range(len(ny.name))])
ny  = ny.assign(rating = [0 for i in range(len(ny.name))])

ny.name = ny.name.str.lower()
ny.name= ny.name.str.replace('’', '', regex=False)
ny.name = ny.name.str.replace('.', '-', regex=False)
ny.name = ny.name.str.replace('!', '-', regex=False)
ny.name = ny.name.str.replace(' ', '-', regex=False)
ny.name = ny.name.str.replace('&', 'and', regex=False)
ny.name = ny.name.str.replace('#', '-no-', regex=False)
ny.name = ny.name.str.replace('--', '-', regex=False)
ny.name = ny.name.str.replace('---', '-', regex=False)
ny.name = ny.name.str.replace('----', '-', regex=False)

In [0]:
def search_business(term, BUSINESS_PATH = BUSINESS_PATH, API_HOST = API_HOST):

    business_path = BUSINESS_PATH + 'search?term=' + term + '&limit=1&location=new-york'
    url = API_HOST + business_path

    #print(url)

    headers = {'Authorization': f"Bearer {API_KEY}"}
    response = requests.get(url, headers=headers)
    return response.json()

In [0]:
for i in range(len(ny.name)):
    response = search_business(ny.name[i])
    #print(response)
    #print(i, ' of ', len(ny.name),' - search term: ', ny.name[i])
    try:
     #print('response term: ', response['businesses'][0]['alias'])
        ny.loc[i, 'alias'] = response['businesses'][0]['alias']
    except (IndexError, KeyError):
    # print('response term: ', "404-Not Found")
    # print('Coordinates cannot be fetched')
    # print('\n')
        continue
    # print(response['businesses'][0]['coordinates'])
    # print('\n')
    ny.loc[i, 'latitude'] = response['businesses'][0]['coordinates']['latitude']
    ny.loc[i, 'longitude'] = response['businesses'][0]['coordinates']['longitude']
    ny.loc[i, 'zipcode'] = response['businesses'][0]['location']['zip_code']
    ny.loc[i, 'review_count'] = response['businesses'][0]['review_count']
    ny.loc[i, 'complete_name'] = response['businesses'][0]['name']
    ny.loc[i, 'rating'] = response['businesses'][0]['rating']

In [0]:
ny.to_csv('productIdMappingWithCoordinates.csv', index=False)

### Loading NY data

In [0]:
productIdMapping = pd.read_csv('./YelpNYC/productIdMapping', header=None, sep='\t')
productIdMapping.columns = ['prod_name', 'prod_id']

In [0]:
productIdMapping

,prod_name,prod_id
0,Snack,0
1,Peppino’s,1
2,Dekalb Restaurant,3
3,Le Parisien,4
4,Blue Spoon Coffee,5
...,...,...
918,Beyoglu,917
919,Benares,902
920,BrisketTown,920
921,Bijan’s,921


In [0]:
productIdMappingWithCoordinates = pd.read_csv('./YelpNYC/productIdMappingWithCoordinates.csv')
#productIdMappingWithCoordinates.columns = ['prod_name_derived', 'prod_id','response_name', 'latitude','longitude']

productIdMappingWithCoordinates.columns = ['prod_name_derived', 'prod_id', 'alias', 'latitude','longitude', 'zipcode', \
                                          'review_count', 'complete_name', 'match']

match_check = productIdMappingWithCoordinates.apply(lambda x: x['prod_name_derived'] in x['alias'], axis = 1)
productIdMappingWithCoordinates['match'] = match_check

In [0]:
productIdMappingWithCoordinates

,prod_name_derived,prod_id,alias,latitude,longitude,zipcode,review_count,complete_name,match
0,snack,0,kuih-cafe-new-york,40.715900,-73.993010,10002,7.0,Kuih Cafe,False
1,peppinos,1,peppinos-brick-oven-pizza-and-restaurant-brook...,40.668221,-73.986936,11215,415.0,Peppino's Brick Oven Pizza & Restaurant,True
2,dekalb-restaurant,3,dekalb-market-hall-brooklyn-2,40.691025,-73.983311,11201,665.0,Dekalb Market Hall,False
3,le-parisien,4,le-parisien-new-york,40.745408,-73.979148,10016,965.0,Le Parisien,True
4,blue-spoon-coffee,5,blue-spoon-coffee-new-york-3,40.708068,-74.007469,10038,188.0,Blue Spoon Coffee,True
...,...,...,...,...,...,...,...,...,...
918,beyoglu,917,beyoglu-new-york,40.775420,-73.956240,10028,1142.0,Beyoglu,True
919,benares,902,benares-new-york-3,40.714150,-74.009209,10007,507.0,Benares,True
920,briskettown,920,hometown-bar-b-que-brooklyn-3,40.674897,-74.016049,11231,1343.0,Hometown Bar-B-Que,False
921,bijans,921,bijans-brooklyn-2,40.688030,-73.986710,11211,575.0,Bijan's,True


In [0]:
metadata = pd.read_csv('./YelpNYC/metadata', header=None, sep='\t')
metadata.columns = ['user_id', 'prod_id', 'rating', 'label', 'date']

In [0]:
metadata

,user_id,prod_id,rating,label,date
0,923,0,3.0,-1,2014-12-08
1,924,0,3.0,-1,2013-05-16
2,925,0,4.0,-1,2013-07-01
3,926,0,4.0,-1,2011-07-28
4,927,0,4.0,-1,2010-11-01
...,...,...,...,...,...
359047,161146,349,5.0,1,2014-02-06
359048,116424,349,5.0,1,2014-01-31
359049,161147,349,5.0,1,2014-01-30
359050,97930,349,5.0,1,2014-01-25


In [0]:
reviewContent = pd.read_csv('./YelpNYC/reviewContent', header=None, sep='\t')
reviewContent.columns = ['user_id', 'prod_id', 'date', 'review']

In [0]:
reviewContent

,user_id,prod_id,date,review
0,923,0,2014-12-08,The food at snack is a selection of popular Gr...
1,924,0,2013-05-16,This little place in Soho is wonderful. I had ...
2,925,0,2013-07-01,ordered lunch for 15 from Snack last Friday. ...
3,926,0,2011-07-28,This is a beautiful quaint little restaurant o...
4,927,0,2010-11-01,Snack is great place for a casual sit down lu...
...,...,...,...,...
358952,161146,349,2014-02-06,"I'm very spoiled with Pizza. Really, I have tr..."
358953,116424,349,2014-01-31,Can't say enough good things about this place....
358954,161147,349,2014-01-30,"Had a great dinner here- fantastic pizza, the ..."
358955,97930,349,2014-01-25,"Great foods and great drinks, they have even p..."


In [0]:
flattened_data = pd.merge(left=metadata, right=reviewContent, how='inner', on=['user_id', 'prod_id', 'date'])
flattened_data = pd.merge(left=flattened_data, right=productIdMappingWithCoordinates, how='left', on=['prod_id'])

In [0]:
flattened_data

,user_id,prod_id,rating,label,date,review,prod_name_derived,alias,latitude,longitude,zipcode,review_count,complete_name,match
0,923,0,3.0,-1,2014-12-08,The food at snack is a selection of popular Gr...,snack,kuih-cafe-new-york,40.71590,-73.99301,10002,7.0,Kuih Cafe,False
1,924,0,3.0,-1,2013-05-16,This little place in Soho is wonderful. I had ...,snack,kuih-cafe-new-york,40.71590,-73.99301,10002,7.0,Kuih Cafe,False
2,925,0,4.0,-1,2013-07-01,ordered lunch for 15 from Snack last Friday. ...,snack,kuih-cafe-new-york,40.71590,-73.99301,10002,7.0,Kuih Cafe,False
3,926,0,4.0,-1,2011-07-28,This is a beautiful quaint little restaurant o...,snack,kuih-cafe-new-york,40.71590,-73.99301,10002,7.0,Kuih Cafe,False
4,927,0,4.0,-1,2010-11-01,Snack is great place for a casual sit down lu...,snack,kuih-cafe-new-york,40.71590,-73.99301,10002,7.0,Kuih Cafe,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358952,161146,349,5.0,1,2014-02-06,"I'm very spoiled with Pizza. Really, I have tr...",emily,emily-brooklyn-2,40.68351,-73.96643,11238,772.0,Emily,True
358953,116424,349,5.0,1,2014-01-31,Can't say enough good things about this place....,emily,emily-brooklyn-2,40.68351,-73.96643,11238,772.0,Emily,True
358954,161147,349,5.0,1,2014-01-30,"Had a great dinner here- fantastic pizza, the ...",emily,emily-brooklyn-2,40.68351,-73.96643,11238,772.0,Emily,True
358955,97930,349,5.0,1,2014-01-25,"Great foods and great drinks, they have even p...",emily,emily-brooklyn-2,40.68351,-73.96643,11238,772.0,Emily,True


In [0]:
flattened_data.to_csv('./YelpNYC/flattened_data_New_York.csv')

### Feature Engineering

In [0]:
data = pd.read_csv('./YelpNYC/flattened_data_New_York.csv')

In [0]:
#Review based

data['review_len'] = data['review'].str.split().str.len()
data['review_char'] = data['review'].str.len()

punct_counter = lambda l1,l2: sum([1 for x in l1 if x in l2])
data['punct_count'] = data['review'].apply(lambda s: punct_counter(s, string.punctuation))

data['uppercase_count'] = data['review'].str.count(r'[A-Z]')


In [0]:
#User based

#data['review_len'] = data['review'].str.split().str.len()

data['review_per_user'] = data.groupby('user_id')['user_id'].transform('size')
data['avg_rating_per_user'] =data.groupby('user_id')['rating'].transform('sum')/data['review_per_user']
data['avg_words_per_user'] = data.groupby('user_id')['review_len'].transform('sum')/data['review_per_user']
data['num_review_per_user_each_day'] = data.groupby(['user_id', 'date'])['user_id'].transform('size')


In [0]:
#Product based

data['review_per_prod'] = data.groupby('prod_id')['prod_id'].transform('size')
data['avg_rating_per_prod'] =data.groupby('prod_id')['rating'].transform('sum')/data['review_per_prod']
data['avg_words_per_prod'] = data.groupby('prod_id')['review_len'].transform('sum')/data['review_per_prod']
data['num_review_per_prod_each_day'] = data.groupby(['prod_id', 'date'])['prod_id'].transform('size')

In [0]:
data.head()

,Unnamed: 0,user_id,prod_id,rating,label,date,review,prod_name_derived,alias,latitude,...,punct_count,uppercase_count,review_per_user,avg_rating_per_user,avg_words_per_user,max_review_per_user_each_day,review_per_prod,avg_rating_per_prod,avg_words_per_prod,max_review_per_prod_each_day
0,0,923,0,3.0,-1,2014-12-08,The food at snack is a selection of popular Gr...,snack,kuih-cafe-new-york,40.7159,...,6,6,39,4.435897,63.717949,2,210,4.009524,87.680952,1
1,1,924,0,3.0,-1,2013-05-16,This little place in Soho is wonderful. I had ...,snack,kuih-cafe-new-york,40.7159,...,10,6,1,3.000000,52.000000,1,210,4.009524,87.680952,1
2,2,925,0,4.0,-1,2013-07-01,ordered lunch for 15 from Snack last Friday. ...,snack,kuih-cafe-new-york,40.7159,...,5,4,2,4.000000,36.500000,2,210,4.009524,87.680952,1
3,3,926,0,4.0,-1,2011-07-28,This is a beautiful quaint little restaurant o...,snack,kuih-cafe-new-york,40.7159,...,14,8,1,4.000000,90.000000,1,210,4.009524,87.680952,1
4,4,927,0,4.0,-1,2010-11-01,Snack is great place for a casual sit down lu...,snack,kuih-cafe-new-york,40.7159,...,18,10,5,3.600000,149.400000,1,210,4.009524,87.680952,2


In [0]:
temp = data[data['label'] == 1]['review']
b_temp = ' '.join(temp)

In [0]:
c_temp.lower()

'tiny little place, but very good food. pastitsio was especially good. i had the braised lamb sandwich and was one of the best sandwiches of my life. do yourself a favour and try this place! friendly service and cosy atmosphere. this spot is close to my job so i decided to check it out for lunch. the prices are just about what you would expect for soho/"south village" sit-down lunch prices; my meal came out to $20 for a mint lemonade and a decent-sized salad. i ordered what i think may have been a special, a roasted chicken & vegetable salad. the chicken was unbelievably dry, to the point where i had tough time chewing without choking a bit. it was served on a bed of arugula which could have chopped a little finer as it was in full, hard to eat pieces. when i eat a salad, i expect it to be cut down to make for easier bites. not necessarily a chopped salad, just pieces that will actually fit in your mouth without chopping it up yourself. basically, i don\'t want to use a knife to eat a 

In [0]:
#n-gram based

#+1 ngrams
text = ' '.join(data[data['label'] == 1]['review'])
tokens = nltk.word_tokenize(text)
pos_bigrams = nltk.bigrams(tokens)
pos_bigrams_freq = nltk.FreqDist(pos_bigrams)
pos_bigrams_freq.most_common()

#-1 ngrams
text = ' '.join(data[data['label'] == -1]['review'])
tokens = nltk.word_tokenize(text)
neg_bigrams = nltk.bigrams(tokens)
neg_bigrams_freq = nltk.FreqDist(neg_bigrams)
neg_bigrams_freq.most_common()


[(('.', 'I'), 25828),
 (('.', 'The'), 25395),
 ((',', 'and'), 13198),
 (('!', '!'), 12874),
 (('and', 'the'), 10986),
 ((',', 'but'), 10597),
 (('of', 'the'), 9890),
 (('in', 'the'), 8516),
 (('.', 'We'), 7980),
 ((',', 'the'), 7558),
 (('this', 'place'), 7491),
 (('.', 'It'), 7076),
 (('it', 'was'), 6816),
 (('the', 'food'), 6433),
 ((',', 'I'), 6398),
 (('and', 'I'), 5752),
 (('on', 'the'), 5622),
 (('for', 'a'), 5455),
 (('it', "'s"), 4942),
 (('the', 'best'), 4813),
 (('for', 'the'), 4780),
 (('I', 'was'), 4749),
 (('to', 'the'), 4685),
 (('!', 'I'), 4300),
 (('I', 'have'), 4298),
 (('do', "n't"), 4253),
 (('I', "'ve"), 4238),
 (('I', 'had'), 4209),
 (('is', 'a'), 4195),
 (('to', 'be'), 4024),
 (('.', 'This'), 4015),
 (('with', 'a'), 3885),
 (('The', 'food'), 3785),
 (('it', '.'), 3771),
 (('.', 'My'), 3709),
 (('food', 'is'), 3702),
 (('I', "'m"), 3690),
 (('with', 'the'), 3619),
 ((')', '.'), 3528),
 (('at', 'the'), 3517),
 (('.', 'They'), 3507),
 (('was', 'a'), 3430),
 (('place'

In [0]:
pos_bigrams_freq.most_common(100)

[(('.', 'I'), 342817),
 (('.', 'The'), 335256),
 ((',', 'and'), 194125),
 ((',', 'but'), 173070),
 (('of', 'the'), 137165),
 (('and', 'the'), 126466),
 (('.', 'It'), 113324),
 (('it', 'was'), 107892),
 ((',', 'I'), 104722),
 (('in', 'the'), 104719),
 (('.', 'We'), 97825),
 ((',', 'the'), 92076),
 (('on', 'the'), 83220),
 (('!', '!'), 79421),
 (('this', 'place'), 78767),
 (('it', "'s"), 77743),
 (('and', 'I'), 73590),
 (('I', 'was'), 71292),
 (('for', 'a'), 66848),
 (('for', 'the'), 63158),
 ((')', '.'), 62420),
 (('I', "'ve"), 61973),
 (('with', 'a'), 60487),
 (('I', "'m"), 59682),
 (('to', 'the'), 59216),
 (('I', 'had'), 58413),
 (('do', "n't"), 58197),
 (('was', 'a'), 57970),
 (('the', 'food'), 56963),
 (('with', 'the'), 55085),
 (('is', 'a'), 51833),
 (('it', '.'), 50762),
 (('.', 'My'), 50612),
 ((',', 'it'), 49529),
 (('.', 'This'), 49178),
 (('to', 'be'), 48406),
 (('.', 'They'), 47847),
 (('the', 'best'), 47430),
 (('did', "n't"), 46260),
 (('had', 'the'), 45579),
 ((',', 'which